In [1]:
#load fetch_ml is depreciated now pass through fetch fetch_openml
from sklearn.datasets import fetch_openml
import numpy as np
import pandas as pd
from sklearn import neighbors
from sklearn.model_selection import train_test_split
#load drive lib
#scoring lib
from mlxtend.evaluate import bias_variance_decomp

In [2]:
#there is a bug on os x with python version > 3.6 with ssl certificate. 
#https://github.com/scikit-learn/scikit-learn/issues/10201
#workaround above 
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
#loading minst form the cloud
mnist = fetch_openml('mnist_784', version=1)


In [4]:
minstFrame = pd.DataFrame(mnist.data)
minstFrame.describe()
mnistFrameTarget = pd.DataFrame(mnist.target)

# Le dataset principal qui contient toutes les images
#print (mnist.data.shape)

# Le vecteur d'annotations associé au dataset (nombre entre 0 et 9)
#print (mnist.target.shape)
#print(mnist.data)

In [25]:
#reduction de la taille du dataset
#generation de 20k index aléatoires et compris entre 1 et 70k
# pour des raisons assez obscure le filtre par entier ne fonctionne pas en local sur le ndarray comme sur colab
# on utilise un workaround avec pandas et loc
sample = np.random.randint(70000, size=50000)
#on sample 20k vecteurs depuis data
data = minstFrame.loc[list(sample)]
target = mnistFrameTarget.loc[list(sample)]

In [6]:
#on split le dataset et le set de resulat pour l'enrtainement et le test avec un classique 80/20

xtrain, xtest, ytrain, ytest = train_test_split(data, target, train_size=0.8)

In [7]:
#loading de la lib knn
from sklearn import neighbors
#n_neighbours est le nombre de voisin autour du point pris pour calculer la classification du future point
#dans l'exemple concernée soit bleu soit rouge
model = neighbors.KNeighborsClassifier(n_neighbors=5)
#A column-vector y was passed when a 1d array was expected
model.fit(data.to_numpy(), target.values.ravel())

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [34]:
#test du model, il attend un tableau 
model.predict([xtest.iloc[100]])

array(['9'], dtype=object)

In [35]:
#pour visualiser le vecteur en question sous forme de tableau d'int de nuances de gris 
xtest.iloc[100]

pixel1      0.0
pixel2      0.0
pixel3      0.0
pixel4      0.0
pixel5      0.0
           ... 
pixel780    0.0
pixel781    0.0
pixel782    0.0
pixel783    0.0
pixel784    0.0
Name: 52123, Length: 784, dtype: float64

In [36]:
#check error for the current model 
error = model.score(xtest, ytest)
print(1 - error)
a = range(2, 15, 1)
print(a[2])

0.019000000000000017
4


In [ ]:
#check where the model is the most accurate by tweaking the amount of neighbours from 2 to 15
errors = []
for k in range(2, 15):
  print(k)
  newmodel = neighbors.KNeighborsClassifier(n_neighbors=k)
  newmodel.fit(xtrain, ytrain)
  errors.append((1-newmodel.score(xtest, ytest))*100)

#print plot error rate for each neighbours size

In [ ]:
print(errors)
import matplotlib.pyplot as plt
#plt.
plt.plot(range(2,15), errors, 'o-')
plt.show()


In [7]:
sample = np.random.randint(70000, size=50000)
#on sample 20k vecteurs depuis data
data = minstFrame.loc[list(sample)]
target = mnistFrameTarget.loc[list(sample)]

print(target)
target.drop(list)

[54506, 44118, 66782, 30702, 29622, 24635, 63085, 21294, 5530, 68125, 35321, 50294, 2242, 30321, 22400, 33119, 4008, 23153, 45430, 15962, 11816, 69277, 47029, 43679, 15807, 18547, 40704, 53773, 37925, 30250, 49464, 47233, 56412, 18868, 54240, 22013, 7830, 27552, 53020, 34805, 17923, 39779, 52166, 26893, 46427, 49947, 63407, 18772, 43715, 15424, 54817, 41907, 57717, 46666, 37692, 48380, 19259, 24729, 37185, 30604, 13966, 69982, 42471, 36675, 30798, 66525, 32218, 30225, 55281, 56325, 55208, 69219, 54312, 53884, 21927, 47782, 30276, 58832, 4603, 4917, 43236, 54134, 21278, 56930, 27763, 66193, 11908, 33761, 48715, 44694, 64599, 69381, 981, 20074, 67680, 44149, 43705, 58923, 41130, 11613, 63109, 23895, 4148, 49654, 67261, 29615, 56399, 9298, 39051, 10558, 35444, 49401, 52531, 25832, 34287, 2892, 11810, 31238, 42345, 66291, 20417, 1307, 61663, 53302, 68183, 14347, 28759, 27647, 11716, 23162, 9713, 26358, 43927, 6332, 23339, 34930, 68656, 37690, 34994, 16199, 11818, 5221, 13524, 16783, 6935, 

In [9]:
#take out of the loop the data for testing

sample = np.random.randint(70000, size=10000)
#it is necessary to replace 0 by 1 in x to avoid issue while scoring bias and var
data = minstFrame.loc[list(sample)]
data = data.sort_index().replace(0, 1).to_numpy()
target = mnistFrameTarget["class"].astype(int).loc[list(sample)]
target = target.sort_index().to_numpy()


#fixe the test dataset for all the model
Xtrain, xtest, Ytrain, ytest = train_test_split(data, target, train_size=0.8)

#dropping the indexes used for test set which is fixed whatever the model is 
minstFrame.drop(list(sample)).sort_index()
mnistFrameTarget.drop(list(sample)).sort_index()


#need to drop those index from dataframe 
for size in range(1000, 12000, 1000):
    performanceFrame = pd.DataFrame(data={"r2": [], "loss": [], "bias": [], "var": [], "knn": [], "size": []})
    print(size)
    #dataset is now 60k objects
    sample = np.random.randint(60000, size=size)
    data = minstFrame.loc[list(sample)].sort_index().replace(0,1).to_numpy()
    target = mnistFrameTarget["class"].astype(int).loc[list(sample)].sort_index().to_numpy()
    xtrain, xtest2, ytrain, ytest2 = train_test_split(data, target, train_size=0.8)
    print(xtrain)
    print(ytrain)
    for neighbours in range(2, 11, 2):
        model = neighbors.KNeighborsClassifier(n_neighbors=neighbours)
        print("neighbours", neighbours)
        #A column-vector y was passed when a 1d array was expected
      
        #loss can be mse or 0-1_loss
        #loss, avg_bias, avg_var = bias_variance_decomp(model, xtrain.to_numpy(), ytrain.values.ravel(), xtest.to_numpy(), ytest.values.ravel(), loss='0-1_loss', random_seed=123)
        loss, avg_bias, avg_var = bias_variance_decomp(model, xtrain, ytrain.ravel(), xtest, ytest, loss='mse', random_seed=123)
        #model.fit(data.to_numpy(), target.values.ravel())
        #model.fit(xtrain.to_numpy(), ytrain.values.ravel())
        model.fit(xtrain, ytrain.ravel())
        print("after fit")
        #R2 = (1-model.score(xtest.to_numpy(), ytest.values.ravel()))*100
        R2 = (1-model.score(xtest, ytest.ravel()))*100
        print("after R2", round(R2, 2))
        
        currentPerformance = pd.DataFrame([{'r2': R2, "loss": loss, "var": avg_var, "bias": avg_bias, "size": size, "knn": neighbours}])
        performanceFrame = performanceFrame.append(currentPerformance)      
    #store the result inside a csv per dataset training size
    performanceFrame.to_csv("../PerformanceAudit/model_knn_fixedTest_performance_"+str(size)+".csv", index=False)
print(performanceFrame.describe())

1000
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[6 9 1 3 7 3 4 3 9 6 6 4 9 9 0 7 2 1 6 0 7 7 9 0 6 5 8 9 5 1 2 6 4 9 9 3 2
 8 9 4 2 0 2 9 2 0 6 2 0 4 8 1 1 2 1 1 4 3 2 0 5 3 1 4 7 7 5 1 4 7 2 6 1 4
 8 1 8 4 0 8 3 0 0 7 6 0 7 3 5 9 5 4 7 6 1 7 5 4 6 8 6 3 8 1 6 1 6 9 5 2 5
 3 2 8 4 6 6 7 2 0 6 3 2 8 0 5 4 1 9 9 3 4 4 7 6 8 3 5 1 5 3 6 6 4 4 7 9 7
 0 6 1 8 2 0 7 9 7 0 5 6 3 4 0 2 0 6 4 9 1 6 0 8 0 8 7 1 7 1 4 3 9 3 1 5 1
 1 8 8 2 1 9 0 0 9 7 3 6 6 4 3 6 6 2 1 9 2 1 8 6 7 0 8 2 9 1 3 6 7 8 2 4 3
 4 7 0 4 0 2 9 6 5 5 7 1 9 0 8 4 2 0 2 5 8 5 3 4 8 1 1 1 7 7 7 1 2 8 4 5 7
 0 0 2 8 8 1 5 6 1 5 5 0 1 0 4 3 9 9 7 5 9 7 4 9 7 5 4 9 6 8 7 0 4 0 7 1 3
 4 4 8 6 3 8 3 0 5 8 9 9 4 4 9 8 7 8 6 0 2 0 2 5 1 9 3 9 1 3 4 2 7 9 2 2 9
 1 1 4 6 8 1 6 5 6 9 1 6 5 1 8 9 9 2 9 6 7 2 5 1 6 6 7 9 8 2 7 9 7 5 4 0 6
 4 0 0 5 8 8 6 6 1 9 1 6 6 9 4 8 6 5 4 4 1 1 6 3 4 1 1 6 8 2 1 8 0 8 5 0 1
 5 2 7 0 8 2 4